In [13]:


import pandas as pd
from datetime import datetime

In [14]:
golden = pd.read_csv('golden.csv', skiprows=1)

In [15]:
newest = pd.read_csv('newest.csv', skiprows=1)

In [16]:
newest.columns

Index(['Appointment Date', 'Gap Tracker', 'Notes', 'DeployedDate',
       'uniquememberid', 'cmsmemberid', 'clientmemberid', 'memberfirstname',
       'membermiddlename', 'memberlastname', 'gender', 'birthdate',
       'phonenumber', 'MemberZipCode', 'assessmentyear', 'programidentifier',
       'clientid', 'healthplanname', 'lineofbusinesscode',
       'planbenefitpackagenumber', 'careprioritygroup', 'issuppressed',
       'uniqueprovidergroupid', 'groupidtype', 'parentgroup',
       'providergroupname', 'grouptin', 'uniqueproviderid',
       'providerfirstname', 'providermiddlename', 'providerlastname',
       'nationalproviderid', 'uniquemembergapid', 'conditionidentifier',
       'gaptype', 'category', 'ruleidentifier', 'gapdescription', 'hcccode',
       'hccmodel', 'hccversion', 'Diag', 'Diagtype', 'gapreason',
       'gapreasonclaim', 'gapreasonlab', 'gapreasonrx', 'suspectlikelihood',
       'analyticrundate', 'gapstatus', 'operationsstatus', 'userdefinedfield1',
       'userde

In [17]:
newest['Notes'].unique()

array([' ', nan, 'Updated by pipeline',
       'Updated at 2023-11-24 13:31:45.372989', 'N18.32', 'I82.509',
       'I50.42', 'N18.31', 'I48.0', 'I73.9', 'G20', 'N18.4', 'G30.1',
       'E11.22', 'E11.59', 'E11.8', 'J44.9', 'I50.30', 'G30.9'],
      dtype=object)

In [18]:
prev_list = list(golden[golden['Notes'] == 'Updated by pipeline']['uniquemembergapid'].unique())
prev_list = set(prev_list)

In [19]:
64-47

17

In [20]:
new_list = list(newest[newest['Notes'] == 'Updated by pipeline']['uniquemembergapid'].unique())
new_list = set(new_list)

In [21]:
new_gaps = list(new_list.difference(prev_list))

In [28]:

def add_not_assessed_to_empty_cells(dataframe):
    time_now = datetime.now()
    # Define a custom function to update 'gap_tracker' within each group
    def update_gap_tracker(group):
    # Check if 'A&P' is present in any row of the group
        has_ap = any(group["Gap Tracker"] == "Assessed and Present")
        sec_filter = group['Gap Tracker'].isna()
        # If 'A&P' is present, update 'gap_tracker' to 'Not Assessed' for non-'A&P' rows
        if has_ap:
            group.loc[(group["Gap Tracker"] != "Assessed and Present") & (sec_filter), "Notes"] = f"Updated at {time_now}"
            group.loc[(group["Gap Tracker"] != "Assessed and Present") & (sec_filter), "Gap Tracker"] = "Not Assessed"
        return group


    # Apply the custom function to each group defined by 'unique_member_id'
    dataframe = dataframe.groupby("uniquememberid").apply(update_gap_tracker)
    # Write the updated DataFrame to a new CSV file
    return dataframe

In [29]:
newest['Gap Tracker'] = newest['Gap Tracker'].str.strip()
df = add_not_assessed_to_empty_cells(newest)

/var/folders/7k/25lsblzs7l57pcqjm7dvqlyw0000gn/T/ipykernel_32518/2355357423.py:16: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dataframe = dataframe.groupby("uniquememberid").apply(update_gap_tracker)


In [30]:
tammy  = df[df['uniquememberid'] == 'COD111265595R']

In [31]:
tammy

,Appointment Date,Gap Tracker,Notes,DeployedDate,uniquememberid,cmsmemberid,clientmemberid,memberfirstname,membermiddlename,memberlastname,...,gapreasonlab,gapreasonrx,suspectlikelihood,analyticrundate,gapstatus,operationsstatus,userdefinedfield1,userdefinedfield2,userdefinedfield3,Submitted Date
179,NaN,Not Assessed,Updated at 2023-11-24 13:31:45.372989,04/07/23,COD111265595R,5CA0K96UP39,336M87099,ELIZABETH,M,KENDALL,...,NaN,NaN,NaN,01/30/23,OPEN,OPEN,NaN,NaN,NaN,
180,20230914.0,Assessed and Present,,04/07/23,COD111265595R,5CA0K96UP39,336M87099,ELIZABETH,M,KENDALL,...,NaN,NaN,Very High,08/30/23,CLOSED,CLOSED,NaN,NaN,NaN,YYYY-00-DD 00:00:SS
181,NaN,Not Assessed,Updated at 2023-11-24 13:31:45.372989,04/07/23,COD111265595R,5CA0K96UP39,336M87099,ELIZABETH,M,KENDALL,...,NaN,NaN,Very High,01/29/23,OPEN,OPEN,NaN,NaN,NaN,
182,NaN,Not Assessed,Updated at 2023-11-24 13:31:45.372989,04/07/23,COD111265595R,5CA0K96UP39,336M87099,ELIZABETH,M,KENDALL,...,NaN,NaN,Very High,08/30/23,CLOSED,CLOSED,NaN,NaN,NaN,
183,NaN,Not Assessed,Updated at 2023-11-24 13:31:45.372989,04/07/23,COD111265595R,5CA0K96UP39,336M87099,ELIZABETH,M,KENDALL,...,NaN,NaN,Very High,05/01/23,OPEN,OPEN,NaN,NaN,NaN,
184,NaN,Not Assessed,Updated at 2023-11-24 13:31:45.372989,04/07/23,COD111265595R,5CA0K96UP39,336M87099,ELIZABETH,M,KENDALL,...,NaN,NaN,Very High,01/29/23,OPEN,OPEN,NaN,NaN,NaN,


['76b84d28-1bb7-37c7-935d-ddb0c7b1ded4',
 '18470681-05ef-344f-9373-0406d56d2d3c',
 '52b4e699-72db-3443-99ca-041b0607001d',
 '6083824b-ad00-3929-a9b1-36761ca3d51a',
 'a3f07cd0-62f2-3859-bcc5-03f0bd0656b5',
 'f34d0292-93cf-39d7-939d-5495fdfee3ca',
 'a723e544-eae1-391b-ac31-20ae532c91e1',
 '513191c4-89bc-39b7-b87c-c41381ce8003',
 '7d57e7b8-5222-3273-93f6-b046c87025b9',
 'af52fa21-43ba-3a56-8532-5f5c5a0252d8',
 '23965c89-61d0-3b42-adb9-60d1c88dd069',
 'e5b22dca-3f1b-3248-811c-c7045ff62425',
 '4eeb1554-0cc7-35ce-b6ef-4eaca765a6f2',
 '09037a9a-ad53-38b1-8cbb-4ee6437e78d0',
 '0a0074d2-0a7f-32f9-a538-42d37ce88dfb',
 'c3f248a2-340e-3458-a104-448b60ec47d3',
 '0c2047a5-0c87-30b9-9b9f-698c95819991']

In [22]:
golden[golden['uniquemembergapid'].isin(new_gaps)]

,Appointment Date,Gap Tracker,Notes,DeployedDate,uniquememberid,cmsmemberid,clientmemberid,memberfirstname,membermiddlename,memberlastname,...,gapreasonlab,gapreasonrx,suspectlikelihood,analyticrundate,gapstatus,operationsstatus,userdefinedfield1,userdefinedfield2,userdefinedfield3,Submitted Date
46,20230208.0,Assessed and Present,NaN,04/07/23,COD15127397NU,4GF9H54QA43,101277997400,WILDA,G,TIMMERMAN,...,NaN,NaN,Very High,09/29/23,OPEN,OPEN,NaN,NaN,NaN,2023-09-23 00:00:00
92,20230630.0,Assessed and Present,,04/07/23,CODBQIWZECNUM,7HG2TG3CN84,108W03601,RUTH,A,POUNDS,...,NaN,NaN,Very High,08/01/23,OPEN,OPEN,NaN,NaN,NaN,2023-09-23 00:00:00
122,20230823.0,Assessed and Present,,04/07/23,CODWXXRIRTPCM,8T20UP2EW28,224M97542,MARSHA,K,BLAUVELT,...,NaN,NaN,Very High,08/01/23,OPEN,OPEN,NaN,NaN,NaN,2023-09-23 00:00:00
326,20230828.0,Assessed and Present,,06/21/23,CODRFTDJ8TT9O,9FH8CE9MR28,601M90245,SARAH,L,EPLER,...,NaN,NaN,Very High,01/29/23,OPEN,OPEN,NaN,NaN,NaN,YYYY-00-DD 00:00:SS
353,20230216.0,Assessed and Present,,04/28/23,COD86501467SQ,2F08CF6DY10,064W06665,IRIS,J,HICKS,...,NaN,NaN,Very High,05/01/23,OPEN,OPEN,NaN,NaN,NaN,2023-09-23 00:00:00
453,20230807.0,Assessed and Present,,04/07/23,COD87279738M8,7CV7HR1PA35,404M66213,CONSUELA,NaN,VANLANDINGHAM,...,NaN,NaN,Low,06/27/23,OPEN,OPEN,NaN,NaN,NaN,2023-09-23 00:00:00
488,20230327.0,Assessed and Present,,04/07/23,COD92368068AB,3XW7JD0PG51,473M65498,MAX,L,REINHART,...,NaN,NaN,Very High,01/29/23,OPEN,OPEN,NaN,NaN,NaN,2023-09-23 00:00:00
545,20230213.0,Assessed and Present,,06/21/23,CODAAE9SN7QQ6,1W71WF1VM35,509W06888,HELENE,F,HEATH,...,NaN,NaN,Very High,01/29/23,OPEN,OPEN,NaN,NaN,NaN,2023-09-23 00:00:00
629,20230620.0,Assessed and Present,,04/07/23,COD92669398T2,8YR8FC0YY61,731M65935,RUTH,E,ABELS,...,NaN,NaN,Medium,06/27/23,OPEN,OPEN,NaN,NaN,NaN,YYYY-00-DD 00:00:SS
637,20230914.0,Assessed and Present,,04/07/23,CODBABQWW5CUC,3KA9NM5YU14,934W06888,MARGARET,J,MILLER,...,NaN,NaN,Very High,03/29/23,OPEN,OPEN,NaN,NaN,NaN,2023-09-23 00:00:00


In [29]:
newest[newest['uniquemembergapid'].isin(new_gaps) & (newest['memberfirstname'] == 'WILDA')].to_dict()

{'Appointment Date': {225: 20230208.0},
 'Gap Tracker': {225: 'Assessed and Present'},
 'Notes': {225: 'Updated by pipeline'},
 'DeployedDate': {225: '04/07/23'},
 'uniquememberid': {225: 'COD15127397NU'},
 'cmsmemberid': {225: '4GF9H54QA43'},
 'clientmemberid': {225: '101277997400'},
 'memberfirstname': {225: 'WILDA'},
 'membermiddlename': {225: 'G'},
 'memberlastname': {225: 'TIMMERMAN'},
 'gender': {225: 'F'},
 'birthdate': {225: '11/02/18'},
 'phonenumber': {225: 2604417445.0},
 'MemberZipCode': {225: 46816.0},
 'assessmentyear': {225: 2023.0},
 'programidentifier': {225: 'IOA'},
 'clientid': {225: 'AET'},
 'healthplanname': {225: 'Aetna Medicare'},
 'lineofbusinesscode': {225: 2.0},
 'planbenefitpackagenumber': {225: '802'},
 'careprioritygroup': {225: 3.0},
 'issuppressed': {225: nan},
 'uniqueprovidergroupid': {225: 202195652.0},
 'groupidtype': {225: 'TIN'},
 'parentgroup': {225: 'Eventus WholeHealth'},
 'providergroupname': {225: 'EXTENDED CARE SPECIALISTS'},
 'grouptin': {225

In [ ]:
Wilda TIMMERMAN '11/02/18'

'211fa5190b31b4a76981c4df439959366106bdebd6421f89477ec68abe9246bc'

In [49]:

# Your hash function
def generate_sha256_hash(firstName, lastName, birth_date_formatted):
    # Convert the date to YYYMMDD format
    birth_date_formatted = pd.to_datetime(birth_date_formatted).strftime('%Y%m%d')
    
    # Convert names to title case
    first_name_titlecase = firstName.title()
    last_name_titlecase = lastName.title()
    
    print(first_name_titlecase + last_name_titlecase + birth_date_formatted)
    # Concatenate and hash
    sha_signature = hashlib.sha256((first_name_titlecase + last_name_titlecase + birth_date_formatted).encode()).hexdigest()
    return sha_signature

In [51]:
import hashlib
generate_sha256_hash('WILDA', 'TIMMERMAN', '19181102')

WildaTimmerman19181102


'46f6e410477701da1425185aa1d53464998f94107edbe88739647f9f26f82b8c'

In [54]:
import pdfkit

# Set the path to the wkhtmltopdf executable (update this with the path on your system)
config = pdfkit.configuration()

# HTML content or file path
# html_content = '<html><body><h1>Hello, PDF!</h1></body></html>'

# Convert HTML to PDF
# pdfkit.from_string(html_content, 'output.pdf', configuration=config)


pdfkit.from_file('test.html', 'output.pdf', configuration=config)


True